# Demo using RAG based on Bing Search

This demo shows how to analyze natural queries using the GPT-3 model, retrieve additional information using a knowledge base, and answer questions.  

For this exercise, you need to create a [Bing Resource](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.Bing%2Faccounts) and input its API Key into the [.env](../.env) file. 

1. Utilizing the Bing Search API: Introducing how to reference search results using the Bing Search API.

In [1]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. How to Use the Bing Search API

### Start with a natural question  
When you need access to data that the LLM might not know, a search engine can be utilized as shown below.

### Step 1: GPT-4o: What do I need to answer this question?

In [2]:
# Define a function to extract necessary information for search
def get_search_info(input_query):
    system_prompt='''
    Answer the following questions as best you can. You have access to the following tools:  
    Web Search: Use the web to find information
    Question: Natural language-based questions that users want to ask through web searches
    Answer: Describes how to effectively answer people's natural language-based questions and creates optimized search keywords needed for use in the Bing Search API. And create optimized search keywords needed for use in the Bing Search API. The answer format must strictly follow the JSON format below.
    {"Keywords": "Specify three sets of keywords information to search for key data that matches the user's question intent."}
    '''
    user_prompt = f'''
    Question: {input_query}
    '''

    # Few-shot example for prompt engineering with Azure OpenAI GPT
    prompt = [{"role":"system", "content":system_prompt},
                {"role":"user", "content":"Question: When is the 2024 South Korean general election and do I get a day off for it?"},
                {"role":"assistant", "content":"""{"Keywords": "2024 South Korean general election date + public holiday status + general election voting policy"}"""},
                {"role":"user","content": user_prompt}
            ]
        
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
        max_tokens=600,
        response_format={"type": "json_object"}
    )

    keywords = json.loads(response.choices[0].message.content)['Keywords']
    
    # If keywords are in list format, join with '+' and return
    if type(keywords) == list:
        keywords = ' + '.join(keywords)

    return json.loads(response.choices[0].message.content)['Keywords']

### Step 2: Search web for more details  

You can find information related to the Bing Search API here:  
https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

In [3]:
# Define a function to fetch search results using the Bing Search API
def get_search_result(keywords):
    subscription_key = os.getenv("BING_SEARCH_KEY")
    endpoint = os.getenv("BING_SEARCH_ENDPOINT") + "/v7.0/search"

    query = keywords

    # Construct a request
    mkt = 'ko_KR'
    params = { 'q': query, 'mkt': mkt }
    headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)

    context = ''
    for result in response.json()['webPages']['value']:
        context += result['snippet'] + '\n'

    return context

In [7]:
# Define a function to generate answers based on search results
def generate_answer(input_query, context):
    system_prompt = """You are an agent who answers the user's questions based on the data provided."""
    user_prompt = f"""Question: {input_query}
    Web Search Results: {context}
    Answer: 
    """

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user","content": user_prompt}
        ],
        max_tokens=600
    )

    return response.choices[0].message.content

In [8]:
# Define a function to process user input and output results
def get_input_query(input_query):
    # Print the input query
    print("-"*80)
    print("Search Query: " + input_query)
    print("-"*80)

    # Call the function to extract search keywords
    keywords = get_search_info(input_query)
    print("Search Keywords: " + keywords)
    print("-"*80)

    # Call the function to fetch search results
    context = get_search_result(keywords)
    print("Search Results: " + context)
    print("-"*80)

    # Call the function to generate an answer for the user
    answer = generate_answer(input_query, context)
    print("Final Answer: " + answer)
    print("-"*80)

Replace the sentence below with a natural language query for the Answer Engine and execute it.

In [11]:
input_query = "When is Buddha's Birthday in 2025? Is it a public holiday? If so, when is the substitute holiday?"
get_input_query(input_query)

--------------------------------------------------------------------------------
Search Query: When is Buddha's Birthday in 2025? Is it a public holiday? If so, when is the substitute holiday?
--------------------------------------------------------------------------------
Search Keywords: Buddha's Birthday 2025 date South Korea + public holiday status + substitute holiday policy 2025
--------------------------------------------------------------------------------
Search Results: Since March 2023, South Korean workers will enjoy an extra day off if either Buddha's Birthday or Christmas overlaps with a Saturday or Sunday. Buddha's birthday has been a public holiday in South Korea since 1975, with the number of followers of Buddhism in South Korea estimated at 10 million.
South Korea Public Holidays 2025. This page contains a national calendar of all 2025 public holidays. These dates may be modified as official changes are announced, so please check back regularly for updates.
Buddha’s B

In [12]:
input_query = "When is the Lunar New Year in 2025, and how many days off are in the holiday period?"
get_input_query(input_query)

--------------------------------------------------------------------------------
Search Query: When is the Lunar New Year in 2025, and how many days off are in the holiday period?
--------------------------------------------------------------------------------
Search Keywords: Lunar New Year 2025 date + South Korea holiday calendar 2025 + Lunar New Year public holiday duration
--------------------------------------------------------------------------------
Search Results: Korean New Year (Seollal) (January 28-30, 2025) Significance: Seollal marks the beginning of the lunar calendar year. It is one of South Korea’s most important traditional holidays, emphasizing family unity and ancestral remembrance.
South Korea Public Holidays 2025. This page contains a national calendar of all 2025 public holidays. These dates may be modified as official changes are announced, so please check back regularly for updates.
Seollal 2025, 2026 and 2027. Seollal, or Korean Lunar New Year, is a major holid